In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ae6db79644779d1b2d580aeff181345deb552f53f05ab78f25bf1272e7cde4cd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("spark").getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
bank=spark.sparkContext.textFile("drive/My Drive/Colab Notebooks/bank.txt")

Mounted at /content/drive/


In [4]:
bank.collect()

['SBI00042\t123876122\t23-10-2021\tC\t250',
 'SBI00042\t123876122\t23-10-2021\tD\t250',
 'SBI00042\t123876122\t23-10-2021\tC\t250',
 'SBI00042\t123872822\t23-10-2021\tC\t1250',
 'SBI00042\t123876122\t23-10-2021\tD\t250',
 'SBI00042\t123823822\t23-10-2021\tC\t10250',
 'SBI00042\t123876171\t23-10-2021\tD\t25000',
 'SBI00042\t123872322\t23-10-2021\tC\t12500',
 'SBI00042\t123876142\t23-10-2021\tD\t2500',
 'SBI00042\t123872829\t23-10-2021\tC\t100',
 'CAN00123\t720127541\t03-1-2010\tC\t250',
 'CAN00123\t123872822\t23-10-2021\tC\t1250',
 'CAN00123\t123876122\t23-10-2019\tD\t250',
 'CAN00123\t123456789\t23-11-2020\tC\t10250',
 'CAN00123\t123456789\t23-12-2020\tC\t10250',
 'CAN00123\t123456789\t23-10-2020\tC\t10250',
 'CAN00123\t123876171\t13-10-2005\tD\t25000',
 'SBI00042\t123872322\t15-11-2012\tC\t12500',
 'SBI00042\t123876142\t06-01-2006\tD\t2500',
 'SBI00042\t123872829\t30-11-2001\tC\t10000',
 'SBI00042\t123872829\t30-07-2001\tC\t10000',
 'SBI00042\t123872829\t30-01-2001\tC\t10000',
 'ICI00

In [5]:
#Count unique number of customers
bank.map(lambda d:d.split('\t')[1]).distinct().count()

15

In [6]:
#Count unique number of Bank ID
bank.map(lambda d:d.split('\t')[0]).distinct().count()

3

In [7]:
#Count unique number of customers per Bank ID
bank.map(lambda d:(d.split('\t')[0],d.split('\t')[1])).distinct().map(lambda d:(str(d).split(',')[0],1)).reduceByKey(lambda a,b:a+b).collect()

[("('SBI00042'", 7), ("('ICI00072'", 6), ("('CAN00123'", 6)]

In [8]:
#Number of transactions for given Account Number
bank.map(lambda d:(d.split('\t')[1],1)).reduceByKey(lambda a,b:a+b).collect()

[('123876122', 6),
 ('123872822', 2),
 ('123876171', 2),
 ('123876142', 4),
 ('123823122', 1),
 ('123809822', 1),
 ('123376422', 1),
 ('123802671', 1),
 ('123823822', 2),
 ('123872322', 3),
 ('123872829', 6),
 ('720127541', 2),
 ('123456789', 5),
 ('123822322', 1),
 ('123830822', 1)]

In [9]:
#Number of credit transactions for given Account Number in a given year
def checkCredit(d):
  if 'C' in d:
    return True
bank.map(lambda d:(d.split('\t')[1]+d.split('\t')[3])).filter(lambda d:checkCredit(d)).map(lambda d:(d.split('C')[0],1)).reduceByKey(lambda a,b:a+b).collect()

[('123876122', 2),
 ('123872822', 2),
 ('123823122', 1),
 ('123809822', 1),
 ('123823822', 2),
 ('123872322', 3),
 ('123872829', 6),
 ('720127541', 2),
 ('123456789', 4),
 ('123822322', 1),
 ('123830822', 1)]

In [10]:
#Get sum of sum of debit amount for an account
def checkDebit(d):
  if 'D' in d:
    return True
bank.filter(lambda d:(checkDebit(d.split('\t')[3]+d.split('\t')[4]))).map(lambda d:(d.split('\t')[1],int(d.split('\t')[4]))).reduceByKey(lambda a,b:a+b).collect()

[('123876122', 1000),
 ('123876171', 50000),
 ('123876142', 10000),
 ('123376422', 250),
 ('123802671', 25000),
 ('123456789', 25000)]

In [11]:
#Get sum of debit and credit amount of an account

def checkCredit(d):
  return d=='C'
bank.filter(lambda d:checkCredit(d.split('\t')[3])).map(lambda d:(d.split('\t')[1],int(d.split('\t')[4]))).reduceByKey(lambda a,b:a+b).collect()

[('123876122', 500),
 ('123872822', 2500),
 ('123823122', 250),
 ('123809822', 1250),
 ('123823822', 20500),
 ('123872322', 37500),
 ('123872829', 40200),
 ('720127541', 500),
 ('123456789', 82000),
 ('123822322', 10250),
 ('123830822', 12500)]

In [12]:
#Checking for debited amount for an account
def checkDebit(d):
  return d=='D'
bank.filter(lambda d:checkDebit(d.split('\t')[3])).map(lambda d:(d.split('\t')[1],int(d.split('\t')[4]))).reduceByKey(lambda a,b:a+b).collect()

[('123876122', 1000),
 ('123876171', 50000),
 ('123876142', 10000),
 ('123376422', 250),
 ('123802671', 25000),
 ('123456789', 25000)]

In [13]:
#Get the number of transactions per a year for an account
bank.map(lambda d:(d.split('\t')[1]+" "+d.split('\t')[2],1)).reduceByKey(lambda a,b:a+b).collect()

[('123823822 23-10-2021', 1),
 ('123876122 23-10-2019', 1),
 ('123876142 06-01-2006', 2),
 ('123872829 30-01-2001', 2),
 ('123809822 23-10-2021', 1),
 ('123802671 23-10-2021', 1),
 ('123830822 23-10-2021', 1),
 ('123876142 11-04-2000', 1),
 ('123456789 03-10-2011', 1),
 ('123876122 06-08-2015', 1),
 ('123823822 23-10-2020', 1),
 ('123876122 23-10-2021', 4),
 ('123872822 23-10-2021', 2),
 ('123876171 23-10-2021', 1),
 ('123872322 23-10-2021', 1),
 ('123876142 23-10-2021', 1),
 ('123872829 23-10-2021', 1),
 ('720127541 03-1-2010', 2),
 ('123456789 23-11-2020', 1),
 ('123456789 23-12-2020', 1),
 ('123456789 23-10-2020', 1),
 ('123876171 13-10-2005', 1),
 ('123872322 15-11-2012', 2),
 ('123872829 30-11-2001', 1),
 ('123872829 30-07-2001', 1),
 ('123823122 23-10-2021', 1),
 ('123376422 23-10-2021', 1),
 ('123822322 23-10-2021', 1),
 ('123872829 21-07-2002', 1),
 ('123456789 13-10-2005', 1)]